# Chapitre 1

Ce Notebook est une brève introduction aux échecs informatiques modernes. Les moteurs d'échecs utilisant l'intelligence artificielle basée sur l'apprentissage profond (deep learning) ont eu un impact considérable sur la communauté des joueurs d'échecs. AlphaZero, le monstre d'échecs développé par DeepMind, la filiale de recherche de Google, a soudain semblé capable de jouer de belles parties d'échecs, presque semblables à celles d'un humain. Certaines parties montraient des sacrifices de pièces sans gain tactique immédiat, mais des avantages positionnels à long terme que les moteurs d'échecs courants étaient incapables de trouver. Nous commencerons par examiner les méthodes de recherche utilisées par les moteurs d'échecs. Nous apprenons comment les moteurs d'échecs "classiques" utilisent la recherche alpha-bêta, et comment la recherche par arbre de Monte-Carlo offre une alternative pour les échecs et d'autres jeux plus difficiles tels que le Shogi ou le Go. À titre d'exemple, nous mettrons en œuvre la recherche alpha-bêta et la recherche arborescente de Monte-Carlo et nous les testerons avec une position d'échecs simple.

Les échecs se composent de tactique et de stratégie. Pour la tactique, vous calculez les coups et les réponses et vous regardez si l'une des séquences de coups conduit à un avantage ou à un désavantage forcé. Il s'agit essentiellement d'une recherche parmi toutes les variantes possibles. La stratégie consiste à juger une position sans trop de calculs. Ici nous nous concentrons sur la tactique.

## Algorithmes de recherche
### Algorithme minmax

On commence par implémenter l'algorithme minimax. Ce code implémente un joueur d'échecs artificiel basé sur un algorithme de recherche appelé minimax avec élagage alpha-bêta. Après une importation de la bibliothèque d'échecs (chess), on commence par définir une table de valeurs pour évaluer la position des pièces sur l'échiquier. Ici c'est très simplifier : plus la pièce est au centre, meilleur est sa postion et inversement.

In [16]:
import chess

In [17]:
pieceSquareTable = [
  [ -50,-40,-30,-30,-30,-30,-40,-50 ],
  [ -40,-20,  0,  0,  0,  0,-20,-40 ],
  [ -30,  0, 10, 15, 15, 10,  0,-30 ],
  [ -30,  5, 15, 20, 20, 15,  5,-30 ],
  [ -30,  0, 15, 20, 20, 15,  0,-30 ],
  [ -30,  5, 10, 15, 15, 10,  5,-30 ],
  [ -40,-20,  0,  5,  5,  0,-20,-40 ],
  [ -50,-40,-30,-30,-30,-30,-40,-50 ] ]

Ensuite on définit une fonction eval(board) qui évalue la position actuelle de l'échiquier en attribuant des valeurs aux pièces en fonction de leur type et de leur position.

In [18]:
# Définition d'une fonction pour évaluer la position actuelle de l'échiquier
def eval(board):
    # Initialisation des scores pour les blancs et les noirs
    scoreWhite = 0
    scoreBlack = 0
    
    # Parcours de chaque case de l'échiquier
    for i in range(0, 8):
        for j in range(0, 8):
            # Obtention de la case (i, j) sur l'échiquier
            squareIJ = chess.square(i, j)
            # Obtention de la pièce sur la case (i, j)
            pieceIJ = board.piece_at(squareIJ)
            
            # Évaluation de la pièce obtenue
            if str(pieceIJ) == "P":  # Si la pièce est un pion blanc
                scoreWhite += (100 + pieceSquareTable[i][j])  # Ajoute la valeur du pion au score des blancs
            if str(pieceIJ) == "N":  # Si la pièce est un cavalier blanc
                scoreWhite += (310 + pieceSquareTable[i][j])  # Ajoute la valeur du cavalier au score des blancs
            if str(pieceIJ) == "B":  # Si la pièce est un fou blanc
                scoreWhite += (320 + pieceSquareTable[i][j])  # Ajoute la valeur du fou au score des blancs
            if str(pieceIJ) == "R":  # Si la pièce est une tour blanche
                scoreWhite += (500 + pieceSquareTable[i][j])  # Ajoute la valeur de la tour au score des blancs
            if str(pieceIJ) == "Q":  # Si la pièce est une reine blanche
                scoreWhite += (900 + pieceSquareTable[i][j])  # Ajoute la valeur de la reine au score des blancs
            
            if str(pieceIJ) == "p":  # Si la pièce est un pion noir
                scoreBlack += (100 + pieceSquareTable[i][j])  # Ajoute la valeur du pion au score des noirs
            if str(pieceIJ) == "n":  # Si la pièce est un cavalier noir
                scoreBlack += (310 + pieceSquareTable[i][j])  # Ajoute la valeur du cavalier au score des noirs
            if str(pieceIJ) == "b":  # Si la pièce est un fou noir
                scoreBlack += (320 + pieceSquareTable[i][j])  # Ajoute la valeur du fou au score des noirs
            if str(pieceIJ) == "r":  # Si la pièce est une tour noire
                scoreBlack += (500 + pieceSquareTable[i][j])  # Ajoute la valeur de la tour au score des noirs
            if str(pieceIJ) == "q":  # Si la pièce est une reine noire
                scoreBlack += (900 + pieceSquareTable[i][j])  # Ajoute la valeur de la reine au score des noirs
    
    # Retourne la différence entre le score des blancs et celui des noirs
    return scoreWhite - scoreBlack


Un jeu de somme nulle est un jeu où la somme des gains et des pertes de tous les joueurs est égale à 0. Cela signifie donc que le gain de l'un constitue obligatoirement une perte pour l'autre. Par exemple si l'on définit le gain d'une partie d'échecs comme 1 si on gagne, 0 si la partie est nulle et -1 si on perd, le jeu d'échecs est un jeu à somme nulle ([Wikipédia 23/04/2024](https://fr.wikipedia.org/wiki/Jeu_%C3%A0_somme_nulle)).

Comme l'explique la page [wikipédia](https://fr.wikipedia.org/wiki/Algorithme_minimax) (23/04/2024), l'algorithme minimax (aussi appelé algorithme MinMax) est un algorithme qui s'applique à la théorie des jeux pour les jeux à deux joueurs à somme nulle (et à information complète) consistant à minimiser la perte maximum (c'est-à-dire dans le pire des cas). Pour une vaste famille de jeux, le théorème du minimax de von Neumann assure l'existence d'un tel algorithme, même si dans la pratique il n'est souvent guère aisé de le trouver. 

Il amène l'ordinateur à passer en revue toutes les possibilités pour un nombre limité de coups et à leur assigner une valeur qui prend en compte les bénéfices pour le joueur et pour son adversaire. Le meilleur choix est alors celui qui minimise les pertes du joueur tout en supposant que l'adversaire cherche au contraire à les maximiser (le jeu est à somme nulle).

Il existe différents algorithmes basés sur MinMax permettant d'optimiser la recherche du meilleur coup en limitant le nombre de nœuds visités dans l'arbre de jeu, le plus connu est l'élagage alpha-bêta. En pratique, l'arbre est souvent trop vaste pour pouvoir être intégralement exploré (comme pour le jeu d'échecs ou de go). Seule une fraction de l'arbre est alors explorée.
Dans le cas d'arbres très vastes, comme pour les échecs, une IA (système expert, évaluation par apprentissage à partir d'exemple, etc.) peut servir à élaguer certaines branches sur la base d'une estimation de leur utilité. Nous verrons cela dans l'autre chapitre.

Maintenant qu'on a une manière d'évaluer le plateau, on implémente l'algorithme minimax pour évaluer les coups possibles et choisir le meilleur coup à jouer.

In [19]:
# Initialisation d'un compteur de nœuds
NODECOUNT = 0

# Définition de la fonction minimax pour évaluer les coups possibles
def minimax(board, depth, maximize):
    global NODECOUNT  # Utilisation de la variable globale NODECOUNT
    
    # Si c'est un échec et mat
    if(board.is_checkmate()):
        # Si c'est au tour des blancs, retourne une valeur très négative
        if(board.turn == chess.WHITE):
            return -100000
        # Sinon, retourne une valeur très positive
        else:
            return 1000000
    
    # Si c'est un pat ou si le matériel est insuffisant, retourne 0
    if(board.is_stalemate()) or board.is_insufficient_material():
        return 0
    
    # Si la profondeur de recherche est égale à 0, retourne l'évaluation de la position actuelle
    if depth == 0:
        return eval(board)
    
    # Obtention des coups légaux possibles pour le joueur actuel
    legals = board.legal_moves
    
    # Si on cherche à maximiser le score
    if(maximize):
        bestVal = -9999  # Initialisation de la meilleure valeur à une valeur très négative
        # Parcours de tous les coups légaux
        for move in legals:
            board.push(move)  # Joue le coup
            NODECOUNT += 1  # Incrémente le compteur de nœuds
            # Appel récursif à minimax pour évaluer le coup suivant avec une profondeur réduite
            bestVal = max(bestVal, minimax(board, depth - 1, (not maximize)))
            board.pop()  # Annule le coup pour revenir à l'état précédent
        return bestVal  # Retourne la meilleure valeur trouvée
    
    # Si on cherche à minimiser le score
    else:
        bestVal = 9999  # Initialisation de la meilleure valeur à une valeur très positive
        # Parcours de tous les coups légaux
        for move in legals:
            board.push(move)  # Joue le coup
            NODECOUNT += 1  # Incrémente le compteur de nœuds
            # Appel récursif à minimax pour évaluer le coup suivant avec une profondeur réduite
            bestVal = min(bestVal, minimax(board, depth - 1, (not maximize)))
            board.pop()  # Annule le coup pour revenir à l'état précédent
        return bestVal  # Retourne la meilleure valeur trouvée


Définition d'une fonction getNextMove(depth, board, maximize) qui prend en compte la profondeur de recherche et renvoie le meilleur coup possible pour un joueur donné.


In [20]:
# Définition de la fonction pour obtenir le prochain meilleur coup à jouer
def getNextMove(depth, board, maximize):
    # Obtention des coups légaux possibles pour le joueur actuel
    legals = board.legal_moves
    # Initialisation du meilleur coup et de sa valeur associée
    bestMove = None
    bestValue = -9999  # Initialise la meilleure valeur à une valeur très négative
    
    # Si on cherche à minimiser le score, initialise la meilleure valeur à une valeur très positive
    if not maximize:
        bestValue = 9999
        
    # Parcours de tous les coups légaux
    for move in legals:
        board.push(move)  # Joue le coup
        # Évalue le score associé au coup actuel en utilisant l'algorithme minimax
        value = minimax(board, depth - 1, (not maximize))
        board.pop()  # Annule le coup pour revenir à l'état précédent
        
        # Met à jour le meilleur coup et sa valeur associée en fonction de la stratégie (maximisation ou minimisation)
        if maximize:  # Si on cherche à maximiser le score
            if value > bestValue:  # Si le score actuel est meilleur que le meilleur score trouvé jusqu'à présent
                bestValue = value  # Met à jour la meilleure valeur
                bestMove = move  # Met à jour le meilleur coup
        else:  # Si on cherche à minimiser le score
            if value < bestValue:  # Si le score actuel est meilleur que le meilleur score trouvé jusqu'à présent
                bestValue = value  # Met à jour la meilleure valeur
                bestMove = move  # Met à jour le meilleur coup
    
    # Retourne le meilleur coup trouvé et sa valeur associée
    return (bestMove, bestValue)


Maintenant on peut exécuter des commandes qui simulent un jeu d'échecs avec quelques mouvements (coup du berger) et affichage du meilleur coup calculé par l'algorithme.


In [21]:
board = chess.Board()
board.push_san("e4")
board.push_san("e5")
board.push_san("Qh5")
board.push_san("Nc6")
board.push_san("Bc4")
board.push_san("Nf6")
print(board)

r . b q k b . r
p p p p . p p p
. . n . . n . .
. . . . p . . Q
. . B . P . . .
. . . . . . . .
P P P P . P P P
R N B . K . N R


On exécute notre code pour voir le résultat.

In [22]:
print(getNextMove(4, board, True))
print(NODECOUNT)

(Move.from_uci('h5f7'), 1000000)
1337211


On obtient bien le meilleur coup : dame en f7 délivrant un échec et mat en 1 coup à l'aide du coup du berger. Cependant il y a 1337211 positions évaluées en environ 3min pour trouver ce coup. C'est beaucoup trop pour un jeu d'échecs complet. On va donc implémenter un algorithme plus rapide pour évaluer les coups possibles.

### Algorithme Alpha-Bêta

On peux cependant optimiser cette algorithme sans modifier le résultat en utilisant l'élagage alpha-bêta. L’élagage alpha-bêta (abrégé élagage αβ) est une technique permettant de réduire le nombre de nœuds évalués par l'algorithme minimax. Lors de l'exploration, il n'est pas nécessaire d'examiner les sous-arbres qui conduisent à des configurations dont la valeur ne contribuera pas au calcul du gain à la racine de l'arbre. Dit autrement, l'élagage αβ n'évalue pas des nœuds dont on peut penser, si la fonction d'évaluation est à peu près correcte, que leur qualité sera inférieure à celle d'un nœud déjà évalué. On peut facilement le voir avec l'image suivante :
![](https://upload.wikimedia.org/wikipedia/commons/thumb/6/64/Poda_alfa-beta.svg/2560px-Poda_alfa-beta.svg.png)

On réutilise notre pieceSquareTable et notre fonction eval. On code maintenant Alpha-Bêta :

In [26]:
# Définition d'un compteur de nœuds
NODECOUNT = 0

# Définition de la fonction alpha-beta pour évaluer les coups possibles
def alphaBeta(board, depth, alpha, beta, maximize):
    global NODECOUNT  # Utilisation de la variable globale NODECOUNT
    
    # Si c'est un échec et mat
    if(board.is_checkmate()):
        # Si c'est au tour des blancs, retourne une valeur très négative
        if(board.turn == chess.WHITE):
            return -100000
        # Sinon, retourne une valeur très positive
        else:
            return 1000000
    
    # Si la profondeur de recherche est égale à 0, retourne l'évaluation de la position actuelle
    if depth == 0:
        return eval(board)
    
    # Obtention des coups légaux possibles pour le joueur actuel
    legals = board.legal_moves
    
    # Si on cherche à maximiser le score
    if(maximize):
        bestVal = -9999  # Initialisation de la meilleure valeur à une valeur très négative
        # Parcours de tous les coups légaux
        for move in legals:
            board.push(move)  # Joue le coup
            NODECOUNT += 1  # Incrémente le compteur de nœuds
            # Appel récursif à alphaBeta pour évaluer le coup suivant avec une profondeur réduite
            bestVal = max(bestVal, alphaBeta(board, depth - 1, alpha, beta, (not maximize)))
            board.pop()  # Annule le coup pour revenir à l'état précédent
            alpha = max(alpha, bestVal)  # Met à jour la valeur alpha
            if alpha >= beta:  # Si alpha est supérieur ou égal à beta
                return bestVal  # Effectue une coupure alpha et retourne la meilleure valeur trouvée
        return bestVal  # Retourne la meilleure valeur trouvée
    
    # Si on cherche à minimiser le score
    else:
        bestVal = 9999  # Initialisation de la meilleure valeur à une valeur très positive
        # Parcours de tous les coups légaux
        for move in legals:
            board.push(move)  # Joue le coup
            NODECOUNT += 1  # Incrémente le compteur de nœuds
            # Appel récursif à alphaBeta pour évaluer le coup suivant avec une profondeur réduite
            bestVal = min(bestVal, alphaBeta(board, depth - 1, alpha, beta, (not maximize)))
            board.pop()  # Annule le coup pour revenir à l'état précédent
            beta = min(beta, bestVal)  # Met à jour la valeur beta
            if beta <= alpha:  # Si beta est inférieur ou égal à alpha
                return bestVal  # Effectue une coupure beta et retourne la meilleure valeur trouvée
        return bestVal  # Retourne la meilleure valeur trouvée

On implémente maintenant notre nouvelle fonction getNextMoveAlphaBeta :

In [24]:
# Définition de la fonction pour obtenir le prochain meilleur coup à jouer
def getNextMoveAlphaBeta(depth, board, maximize):
    # Obtention des coups légaux possibles pour le joueur actuel
    legals = board.legal_moves
    
    # Initialisation du meilleur coup et de sa valeur associée
    bestMove = None
    bestValue = -9999  # Initialise la meilleure valeur à une valeur très négative
    
    # Si on cherche à minimiser le score, initialise la meilleure valeur à une valeur très positive
    if not maximize:
        bestValue = 9999
    
    # Parcours de tous les coups légaux
    for move in legals:
        board.push(move)  # Joue le coup
        # Évalue le score associé au coup actuel en utilisant l'algorithme alpha-beta
        value = alphaBeta(board, depth - 1, -10000, 10000, (not maximize))
        board.pop()  # Annule le coup pour revenir à l'état précédent
        
        # Met à jour le meilleur coup et sa valeur associée en fonction de la stratégie (maximisation ou minimisation)
        if maximize:  # Si on cherche à maximiser le score
            if value > bestValue:  # Si le score actuel est meilleur que le meilleur score trouvé jusqu'à présent
                bestValue = value  # Met à jour la meilleure valeur
                bestMove = move  # Met à jour le meilleur coup
        else:  # Si on cherche à minimiser le score
            if value < bestValue:  # Si le score actuel est meilleur que le meilleur score trouvé jusqu'à présent
                bestValue = value  # Met à jour la meilleure valeur
                bestMove = move  # Met à jour le meilleur coup
    
    # Retourne le meilleur coup trouvé et sa valeur associée
    return (bestMove, bestValue)

On peut maintenant faire un test avec la position précédente :

In [27]:
print(getNextMoveAlphaBeta(4, board, True))
print(NODECOUNT)

(Move.from_uci('h5f7'), 1000000)
119034


On obtient bien le même résultat, l'échec et mat avec Dame en f7, cepandant on a évalué 119034 positions en environ 15 secondes au lieu de 1337211. C'est déjà beaucoup mieux mais on peut faire encore mieux avec un autre algorithme de recherche.


Il est difficile de déterminer un niveau ELO précis pour l'algorithme minimax aux échecs car cela dépend de plusieurs facteurs : 
- Profondeur de recherche
- Fonction d'évaluation
- Optimisations



En résumé, la force de l'algorithme minimax aux échecs varie considérablement en fonction de sa mise en œuvre et des optimisations employées, se situant généralement entre 1500 et 2200 ELO pour les versions basiques et pouvant dépasser les 2200 ELO pour les implémentations plus avancées.